In [1]:
import numpy as np
from astropy.modeling import models, fitting
import matplotlib.pyplot as plt
from pathlib import Path
from astropy.io import fits
from astropy.wcs import WCS
import os
import scipy.stats as stats
import json
import pandas as pd
from fitsutil import *

In [2]:
eventPath = Path(
    r"Z:\Astro_BlackCAT\BlackCAT_Calibration_Data\LC_Calibration_Data\y2024-12-12\BC057_Ti_FF_233_HORI_0_0\Analysis\imaging_analysis_20241213\combined_events_th1_200.fits.gz"
)
d = fits.open(eventPath)

In [3]:
print(type(d))

<class 'astropy.io.fits.hdu.hdulist.HDUList'>


In [3]:
noiseShelf, signal = splitEventList(d,700,"sum")

In [4]:
a,b = halfEventList(signal)

In [5]:
a = random_sample_events_list(a,1000)
b = random_sample_events_list(b,1000)

In [6]:
noise = np.array(noiseShelf.data)


In [19]:
a.data = np.append(a.data,noise)
print(len(a.data))

518692
